In [1]:
import numpy as np

##### Создадим рандомный трейнсет из 20 числовых и 1 категориального признака

In [2]:
N = 1000
n_rand_feats = 20

X_real = np.random.randn(N, n_rand_feats)
X_cat = np.random.binomial(10, 0.2, N).reshape(N, 1).astype(str)

In [3]:
X_real.shape, X_cat.shape

((1000, 20), (1000, 1))

In [4]:
X = np.hstack([X_real, X_cat])
X.shape

(1000, 21)

In [5]:
def func(X):
    return X[:, 0] * 32 + X[:, 3] * (-32.45) + X[:, 4] * (-33.45) + X[:, 9] * (122.45) + X[:, 12] * (11.45)

In [6]:
maps = {}
for v, i in enumerate(np.unique(X[:, 20])):
    maps[i] = np.random.randint(20, 200)

maps

{'0': 54,
 '1': 36,
 '2': 23,
 '3': 20,
 '4': 91,
 '5': 155,
 '6': 160,
 '7': 96,
 '8': 133}

In [7]:
cat_tmp = X[:, 20].copy()

for i,k in maps.items():
    cat_tmp[cat_tmp==i] = int(k)

cat_tmp = cat_tmp.astype(int)

##### Забацаем таргет по данным, чтоб было нормально

In [8]:
y = ((func(X[:, :20].astype(float)) * cat_tmp) > 0).astype(int)

In [9]:
np.mean(y)

0.502

##### Обучим катбуст модельку

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=242)

In [12]:
from catboost import CatBoostClassifier

In [13]:
model = CatBoostClassifier(iterations=2, depth=3, cat_features=[20], eval_metric='Accuracy')

In [14]:
fit_model = model.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.8775000	total: 55.1ms	remaining: 55.1ms
1:	learn: 0.8912500	total: 56.7ms	remaining: 0us


In [16]:
fit_model.save_model('/Users/a.safiullin/work/pet/catboost_service/go_app/model.catboost')

In [17]:
y_pred = fit_model.predict_proba(X_test)[:, 1]

In [18]:
y_test.shape

(200,)

In [19]:
y_pred.shape

(200,)

In [20]:
from sklearn.metrics import roc_auc_score

In [21]:
roc_auc_score(y_test, y_pred)

0.9441991168205539

In [22]:
X_test[:, 20:].shape

(200, 1)

In [23]:
X_test[:, :20].astype(float).shape

(200, 20)

In [32]:
X_test[0].astype(float)

array([-0.74739256,  1.7759689 ,  0.9164685 ,  1.69462746,  0.15660069,
       -0.25860515,  0.41270785,  0.33380319,  0.93324519,  0.91686549,
        0.37013758,  1.11828495, -0.04145232,  0.28582875, -1.23362585,
       -0.01187642,  0.61897348,  1.26059207, -1.0499095 , -0.55483756,
        2.        ])

In [33]:
y_test[0]

1

In [34]:
X_test[1].astype(float)

array([ 7.16649064e-01, -4.33761121e-01, -9.97765087e-01,  2.40422338e+00,
        1.63855342e-03, -1.52299574e+00,  1.73964321e-01,  2.00667088e-01,
       -9.70451456e-01, -3.68434446e-01, -6.41791845e-01,  2.85841898e-01,
        1.77142870e+00, -4.91911687e-01, -1.18888478e+00, -3.64192469e-01,
        1.91465730e+00, -1.44569499e+00,  1.39303868e+00, -6.81542112e-01,
        1.00000000e+00])

In [35]:
y_test[1]

0

In [36]:
y_pred[0]

0.8440786283218703

In [37]:
y_pred[1]

0.24511496453349393